# PATH CONFIG

In [1]:
import os

print(os.getcwd())
if not os.getcwd().endswith("app"):
    os.chdir("../app")
    print(os.getcwd())

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

%load_ext autoreload
%autoreload 2
# %matplotlib inline

/home/turbotowerlnx/Documents/Master/TA/TA-Spanish-Esperanto-Translator/notebooks
/home/turbotowerlnx/Documents/Master/TA/TA-Spanish-Esperanto-Translator/app


In [2]:
from src.config import Configuration

CONFIG = Configuration()

# Dataset

In [3]:
df_corpus_clean = pd.read_csv(CONFIG.corpus_path)